<a href="https://colab.research.google.com/github/ArkS0001/LoRA-Adapter/blob/main/lora_adapters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!huggingface-cli login --token

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Flipkart` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Flipkart`


In [1]:
# =========================
# Full End-to-End: LoRA Adapter + Groq API
# =========================

!pip install -q transformers peft datasets accelerate bitsandbytes groq

import os, torch
from getpass import getpass
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from peft import LoraConfig, get_peft_model
from groq import Groq

# --- Secure API key ---
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")
client = Groq(api_key=os.environ["GROQ_API_KEY"])

# --- Step 1: Load base model (small demo model locally, 7B not 17B) ---
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")

# --- Step 2: Add LoRA ---
lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=["q_proj","v_proj"],
                         lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(base_model, lora_config)

# --- Step 3: Tiny dataset ---
data = Dataset.from_dict({
    "text": [
        "Customer A always talks about finance and stocks.",
        "Customer B always talks about legal contracts and law."
    ]
})
def tokenize(batch): return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)
tokenized = data.map(tokenize, batched=True)
train_loader = DataLoader(tokenized, batch_size=1, shuffle=True)

# --- Step 4: Train adapter (quick 1 epoch demo) ---
optimizer = AdamW(model.parameters(), lr=1e-4)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).train()
for batch in train_loader:
    optimizer.zero_grad()
    ids, mask = batch["input_ids"].to(device), batch["attention_mask"].to(device)
    loss = model(input_ids=ids, attention_mask=mask, labels=ids).loss
    loss.backward(); optimizer.step()
    print("Loss:", loss.item())

# --- Step 5: Save adapter & zip ---
out_dir = "customerA_adapter"
model.save_pretrained(out_dir); tokenizer.save_pretrained(out_dir)
!zip -r customerA.zip customerA_adapter > /dev/null

# --- Step 6: Upload adapter to Groq ---
with open("customerA.zip", "rb") as f:
    file_obj = client.files.create(file=f, purpose="fine_tuning")
print("Uploaded File ID:", file_obj.id)

# --- Step 7: Register adapter against Groq base model ---
fine_tune = client.fine_tunings.create(
    input_file_id=file_obj.id,
    name="customerA-finance-adapter",
    type="lora",
    base_model="meta-llama/llama-4-maverick-17b-128e-instruct"
)
print("Registered Adapter Model:", fine_tune.data.fine_tuned_model)

# --- Step 8: Inference with LoRA adapter ---
completion = client.chat.completions.create(
    model=fine_tune.data.fine_tuned_model,
    messages=[
        {"role": "system", "content": "You are a finance-specific assistant."},
        {"role": "user", "content": "Give me today’s stock market summary."},
    ],
)
print("Response:\n", completion.choices[0].message.content)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 8.3 MB/s eta 0:00:00


ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.12/dist-packages/transformers/__init__.py)

In [12]:
# =========================
# Full End-to-End: LoRA Adapter + Groq API (fixed AdamW import)
# =========================

!pip install -q transformers peft datasets accelerate bitsandbytes groq

import os, torch
from getpass import getpass
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.optim import AdamW   # <-- FIXED import
from peft import LoraConfig, get_peft_model
from groq import Groq

# --- Secure API key ---
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")
client = Groq(api_key=os.environ["GROQ_API_KEY"])

# --- Step 1: Load base model (small demo model locally, 7B not 17B) ---
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")

# --- Step 2: Add LoRA ---
lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=["q_proj","v_proj"],
                         lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(base_model, lora_config)

# --- Step 3: Tiny dataset ---
data = Dataset.from_dict({
    "text": [
        "Customer A always talks about finance and stocks.",
        "Customer B always talks about legal contracts and law."
    ]
})
def tokenize(batch): return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)
tokenized = data.map(tokenize, batched=True)
train_loader = DataLoader(tokenized, batch_size=1, shuffle=True)

# --- Step 4: Train adapter (quick 1 epoch demo) ---
optimizer = AdamW(model.parameters(), lr=1e-4)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).train()
for batch in train_loader:
    optimizer.zero_grad()
    ids, mask = batch["input_ids"].to(device), batch["attention_mask"].to(device)
    loss = model(input_ids=ids, attention_mask=mask, labels=ids).loss
    loss.backward(); optimizer.step()
    print("Loss:", loss.item())

# --- Step 5: Save adapter & zip ---
out_dir = "customerA_adapter"
model.save_pretrained(out_dir); tokenizer.save_pretrained(out_dir)
!zip -r customerA.zip customerA_adapter > /dev/null

# --- Step 6: Upload adapter to Groq ---
with open("customerA.zip", "rb") as f:
    file_obj = client.files.create(file=f, purpose="fine_tuning")
print("Uploaded File ID:", file_obj.id)

# --- Step 7: Register adapter against Groq base model ---
fine_tune = client.fine_tunings.create(
    input_file_id=file_obj.id,
    name="customerA-finance-adapter",
    type="lora",
    base_model="meta-llama/llama-4-maverick-17b-128e-instruct"
)
print("Registered Adapter Model:", fine_tune.data.fine_tuned_model)

# --- Step 8: Inference with LoRA adapter ---
completion = client.chat.completions.create(
    model=fine_tune.data.fine_tuned_model,
    messages=[
        {"role": "system", "content": "You are a finance-specific assistant."},
        {"role": "user", "content": "Give me today’s stock market summary."},
    ],
)
print("Response:\n", completion.choices[0].message.content)


Enter your Groq API key: ··········


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 